# Retrieval Augmented Generation

## Setup API clients

In [1]:
import os

import dotenv
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

openai_client = AzureOpenAI(
        api_key=os.getenv("aozai_api_key"),  
        api_version=os.getenv("aoai_llm_api_version"),
        azure_endpoint=os.getenv("aoai_resource_uri")
    )

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=os.getenv("aoai_embedding_model_name"), input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_API_KEY"))

AZURE_SEARCH_FULL_INDEX = "envision-qa-index"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=search_creds)


## Prepare user question

In [ ]:
user_question = "咨询电脑下载腾讯会议软件问题"
user_question_vector = get_embedding(user_question)

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [ ]:
r = search_client.search(
        user_question,
        top=3, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="question_embedding,answer_embedding")],
        query_type="semantic",
        semantic_configuration_name="default")

sources = "\n\n".join([f"[{doc['sourcepage']}]: {doc['content']}\n" for doc in r])

print(sources)

[role_library-28.png]: · Knowledge of industry trends, technologies, and regulations.
· Excellent communication and presentation skills.
Manager of Product Management
Job Title: Manager of Product Management, Contoso Electronics
Job Summary:
The Manager of Product Management is responsible for overseeing the product management team, driving product development and marketing strategy for Contoso Electronics. This individual will be accountable for the successful launch of new products and the implementation of product life-cycle management processes. The Manager of Product Management will collaborate with internal teams, such as engineering, sales, marketing, and finance, as well as external partners, suppliers, and customers to ensure successful product execution.
Responsibilities:
· Lead the product management team and provide guidance on product strategy, design, development, and launch.
· Develop and implement product life-cycle management processes.
· Monitor and analyze industry t

## Send question and documents to the model

In [4]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

A product manager is responsible for overseeing the product management team, driving product development and marketing strategy, and ensuring successful product execution [role_library-28.png]. They collaborate with internal teams such as engineering, sales, marketing, and finance, as well as external partners, suppliers, and customers [role_library-28.png]. They develop and implement product life-cycle management processes, monitor industry trends, develop product marketing plans and go-to-market strategies, research customer needs, collaborate with internal teams, develop pricing strategies and cost models, oversee product portfolio and performance metrics, manage product development budget, and analyze product performance and customer feedback [role_library-28.png]. They should have a bachelor's degree in business, engineering, or a related field and at least 5 years of experience in product management with a proven track record of successful product launches [role_library-28.png]. 